## Dependencies

In [1]:
import glob
import html
import warnings
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate, Add, Average
from googleqa_utilityscript import *
from transformers import DistilBertConfig, DistilBertTokenizer, TFDistilBertModel


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
MODULE_BASE_PATH = '/kaggle/input/distilbert-base-uncased-huggingface/distilbert-base-uncased'
MODULE_PATH = MODULE_BASE_PATH + '/distilbert-base-uncased-tf_model.h5'
VOCAB_PATH = MODULE_BASE_PATH + '/bert-base-uncased-vocab.txt'
CONF_PATH = MODULE_BASE_PATH + '/distilbert-base-uncased-config.json'
model_path_list = glob.glob('/kaggle/input/155distilbert-best/' + '*.h5')
model_path_list.sort()
print('Models to predict:')
print(*model_path_list, sep = "\n")

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict:
/kaggle/input/155distilbert-best/155-DistilBert_model_fold_1.h5
/kaggle/input/155distilbert-best/155-DistilBert_model_fold_2.h5
/kaggle/input/155distilbert-best/155-DistilBert_model_fold_3.h5
/kaggle/input/155distilbert-best/155-DistilBert_model_fold_4.h5
/kaggle/input/155distilbert-best/155-DistilBert_model_fold_5.h5
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
text_features_question = ['question_title', 'question_body']
text_features_answer = ['question_title', 'answer']

test['question_title'] = test['question_title'].apply(html.unescape)
test['question_body'] = test['question_body'].apply(html.unescape)
test['answer'] = test['answer'].apply(html.unescape)

# Model parameters

In [5]:
N_CLASS = len(target_cols)
N_CLASS_QUESTION = len(question_target_cols)
N_CLASS_ANSWER = len(answer_target_cols)
MAX_SEQUENCE_LENGTH = 512
MAX_LENGTH_TITLE = 30
MAX_LENGTH_QUESTION = 479
MAX_LENGTH_ANSWER = 479

## Test set

In [6]:
tokenizer = DistilBertTokenizer.from_pretrained(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays_2(test, text_features_question, tokenizer, MAX_SEQUENCE_LENGTH, 
                               MAX_LENGTH_TITLE, MAX_LENGTH_QUESTION, ignore_first_sep=False)\
        + compute_input_arays_2(test, text_features_answer, tokenizer, MAX_SEQUENCE_LENGTH, 
                                MAX_LENGTH_TITLE, MAX_LENGTH_ANSWER, ignore_first_sep=False)

# Model

In [7]:
module_config = DistilBertConfig.from_pretrained(CONF_PATH)
module_config.output_hidden_states=False

def model_fn():
    input_word_ids_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids_question')
    input_masks_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks_question')
    segment_ids_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids_question')

    input_word_ids_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids_answer')
    input_masks_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks_answer')
    segment_ids_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids_answer')

    module_model = TFDistilBertModel.from_pretrained(MODULE_PATH, config=module_config)
    sequence_output_question = module_model([input_word_ids_question, input_masks_question])
    sequence_output_answer = module_model([input_word_ids_answer, input_masks_answer])

    sequence_output_question = sequence_output_question[0] # The last hidden-state is the first element of the output tuple
    sequence_output_answer = sequence_output_answer[0] # The last hidden-state is the first element of the output tuple
    
    x_question = GlobalAveragePooling1D()(sequence_output_question)
    x_question = Dropout(0.2)(x_question)
    output_question = Dense(N_CLASS_QUESTION, kernel_initializer='glorot_uniform', activation="sigmoid", name="output_question")(x_question)
    
    x_answer = GlobalAveragePooling1D()(sequence_output_answer)
    x_answer = Dropout(0.2)(x_answer)
    output_answer = Dense(N_CLASS_ANSWER, kernel_initializer='glorot_uniform', activation="sigmoid", name="output_answer")(x_answer)
    
    output = Concatenate()([output_question, output_answer])

    model = Model(inputs=[input_word_ids_question, input_masks_question, segment_ids_question, 
                          input_word_ids_answer, input_masks_answer, segment_ids_answer], outputs=output)

    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test) / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.945054,0.680519,0.223466,0.452661,0.603430,0.585105,0.684945,0.652462,0.686694,...,0.928978,0.910574,0.584792,0.963134,0.966040,0.852204,0.011492,0.012787,0.893806,0.935003
1,46,0.898379,0.515714,0.008813,0.754469,0.801327,0.920165,0.585270,0.488293,0.112530,...,0.754160,0.946129,0.669443,0.971998,0.984187,0.892557,0.942592,0.117646,0.069729,0.913829
2,70,0.925720,0.692736,0.019802,0.761818,0.931599,0.955639,0.580630,0.474654,0.161049,...,0.899262,0.932327,0.578349,0.962234,0.961420,0.849369,0.026991,0.034370,0.843774,0.926659
3,132,0.916849,0.409530,0.005792,0.696915,0.759213,0.925504,0.533760,0.425085,0.086380,...,0.769474,0.952722,0.724147,0.969323,0.982379,0.905933,0.662345,0.153191,0.748847,0.908938
4,200,0.930902,0.456727,0.040709,0.832439,0.764657,0.907189,0.625465,0.588965,0.120619,...,0.738745,0.909433,0.654090,0.964612,0.960412,0.821241,0.174312,0.116578,0.743336,0.919729


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.915241,0.580571,0.038515,0.708409,0.791161,0.833835,0.587263,0.501642,0.240340,...,0.835407,0.934091,0.676838,0.967217,0.973583,0.870476,0.508795,0.123997,0.538835,0.926611
std,2812.670060,0.037773,0.146230,0.062280,0.123056,0.104497,0.137836,0.055563,0.094076,0.225773,...,0.076496,0.029083,0.051516,0.012503,0.013393,0.044033,0.329165,0.059075,0.294143,0.019391
min,39.000000,0.789158,0.299687,0.003044,0.216176,0.386931,0.139859,0.486488,0.342731,0.008492,...,0.595166,0.728783,0.523669,0.864043,0.866600,0.655754,0.006900,0.011885,0.033955,0.841455
25%,2572.000000,0.890330,0.455372,0.008928,0.638570,0.740246,0.798841,0.541983,0.428828,0.064810,...,0.773348,0.919740,0.640249,0.961245,0.966681,0.846849,0.147799,0.078548,0.273005,0.915140
50%,5093.000000,0.919782,0.559371,0.014522,0.718843,0.801526,0.887406,0.574939,0.473684,0.145101,...,0.836780,0.940707,0.676568,0.969566,0.977369,0.880173,0.564573,0.122272,0.538924,0.929245
75%,7482.000000,0.944959,0.708206,0.036146,0.791725,0.860225,0.920939,0.626088,0.561118,0.380093,...,0.904077,0.954221,0.712763,0.975307,0.983460,0.903152,0.817499,0.166315,0.816225,0.940422
max,9640.000000,0.985047,0.917458,0.410055,0.974920,0.984204,0.985726,0.746800,0.805190,0.899370,...,0.970717,0.979376,0.810687,0.990334,0.990906,0.956297,0.962005,0.302530,0.990291,0.966429
